# Ejercicio 1: Ajuste de Hiperparámetros con TensorBoard

## Objetivo
Desarrollar un modelo de red neuronal para clasificación binaria ajustando hiperparámetros y visualizar el proceso de entrenamiento usando TensorBoard.

In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import datetime
import os

## 1. Carga y Preprocesamiento de Datos
Primero cargaremos el dataset de SMS spam y realizaremos el preprocesamiento necesario.

In [ ]:
# Cargar los datos
data = pd.read_csv('spam.csv', encoding='latin-1')
texts = data['v2'].values
labels = data['v1'].values

# Codificar las etiquetas
le = LabelEncoder()
labels = le.fit_transform(labels)

# Tokenización y padding
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=100)

# División en train y test
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

## 2. Definición del Modelo Base
Crearemos una función para construir el modelo con parámetros configurables.

In [ ]:
def create_model(learning_rate, architecture, activation):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(5000, 32, input_length=100),
        tf.keras.layers.Dense(architecture[0], activation=activation),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(architecture[1], activation=activation),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    return model

## 3. Configuración de TensorBoard y Entrenamiento
Configuraremos TensorBoard y entrenaremos diferentes modelos con distintas combinaciones de hiperparámetros.

In [ ]:
# Definir los hiperparámetros a probar
learning_rates = [0.001, 0.01, 0.1]
architectures = [(32, 16), (64, 32), (128, 64)]
activations = ['relu', 'tanh']

# Crear directorio para logs
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Entrenar diferentes modelos
for lr in learning_rates:
    for arch in architectures:
        for act in activations:
            # Crear nombre único para cada configuración
            name = f"lr_{lr}_arch_{arch[0]}_{arch[1]}_act_{act}"
            print(f"Training model with configuration: {name}")
            
            # Configurar TensorBoard callback
            tensorboard_callback = TensorBoard(
                log_dir=log_dir + "/" + name,
                histogram_freq=1
            )
            
            # Crear y entrenar el modelo
            model = create_model(lr, arch, act)
            
            model.fit(
                X_train, y_train,
                epochs=10,
                validation_data=(X_test, y_test),
                callbacks=[tensorboard_callback]
            )

## 4. Visualización en TensorBoard
Para visualizar los resultados en TensorBoard, ejecuta el siguiente comando en una celda:

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

## 5. Evaluación del Mejor Modelo
Después de analizar los resultados en TensorBoard, podemos crear y evaluar el modelo con los mejores hiperparámetros encontrados.

In [ ]:
# Crear el mejor modelo (después de analizar resultados)
best_lr = 0.001  # Ajustar según resultados
best_architecture = (64, 32)  # Ajustar según resultados
best_activation = 'relu'  # Ajustar según resultados

best_model = create_model(best_lr, best_architecture, best_activation)

# Entrenar el mejor modelo
history = best_model.fit(
    X_train, y_train,
    epochs=15,
    validation_data=(X_test, y_test)
)

# Evaluar el modelo
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print(f"\nTest accuracy: {test_accuracy:.4f}")